**If you found this helpful, please *upvote* this kerel and the associated dataset.**

This is my original kernel [click me](https://www.kaggle.com/unkownhihi/starter-kernel-with-cnn-model). This kernel has a much more better LB score than the original one and changed a whole lot, so I feel like publishing another one.

Feel free to use my "DeepFake" dataset. It includes  chunks of training data(only include cropped faces) and I will add new ones once in a while. **BEFORE YOU USE THE DATASET ATACHED TO THIS KERNEL, YOU HAVE TO AGREE THE DEEPFAKE COMPETITION RULE.**

Edit: 
1. Add more data

Code for generating dataset:

In [1]:
from mtcnn import MTCNN
import tqdm
import datetime
import smtplib
import os
import cv2
import numpy as np
import sys
import shutil
d_num=sys.argv[1]
if len(d_num)==1:
    a_num = d_num
    d_num='0'+d_num
else:
    a_num=d_num
detector = MTCNN()
def detect_face(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    final = []
    detected_faces_raw = detector.detect_faces(img)
    if detected_faces_raw==[]:
        #print('no faces found')
        return []
    confidences=[]
    for n in detected_faces_raw:
        x,y,w,h=n['box']
        final.append([x,y,w,h])
        confidences.append(n['confidence'])
    if max(confidences)<0.7:
        return []
    max_conf_coord=final[confidences.index(max(confidences))]
    #return final
    return max_conf_coord
def crop(img,x,y,w,h):
    x-=40
    y-=40
    w+=80
    h+=80
    if x<0:
        x=0
    if y<=0:
        y=0
    return cv2.cvtColor(cv2.resize(img[y:y+h,x:x+w],(256,256)),cv2.COLOR_BGR2RGB)
def detect_video(video):
    v_cap = cv2.VideoCapture(video)
    v_cap.set(1, NUM_FRAME)
    success, vframe = v_cap.read()
    vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
    bounding_box=detect_face(vframe)
    if bounding_box==[]:
        count=0
        current=NUM_FRAME
        while bounding_box==[] and count<MAX_SKIP:
            current+=1
            v_cap.set(1,current)
            success, vframe = v_cap.read()
            vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
            bounding_box=detect_face(vframe)
            count+=1
        if bounding_box==[]:
            print('hi')
            return None
    x,y,w,h=bounding_box
    v_cap.release()
    return crop(vframe,x,y,w,h)
test_dir = './dfdc_train_part_' + a_num + '/'
test_video_files = [test_dir + x for x in os.listdir(test_dir)]
os.makedirs('./DeepFake' + d_num,exist_ok=True)
MAX_SKIP=10
NUM_FRAME=150
count=0
for video in tqdm.tqdm(test_video_files):
    try:
        if video=='./dfdc_train_part_'+a_num+'/metadata.json':
            shutil.copyfile(video,'./metadata'+str(a_num)+'.json')
        img_file=detect_video(video)
        os.remove(video)
        if img_file is None:
            count+=1
            continue
        cv2.imwrite('./DeepFake'+d_num+'/'+video.replace('.mp4','').replace(test_dir,'')+'.jpg',img_file)
    except Exception as err:
      print(err)

ModuleNotFoundError: No module named 'mtcnn'

# Install MTCNN

In [2]:
!pip install ../input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl

Processing /kaggle/input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl


# Import Libraries

In [3]:
import pandas as pd
import keras
import os
import numpy as np
from sklearn.metrics import log_loss
from keras import Model,Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split
import cv2
from tqdm.notebook import tqdm
import glob
import torch
from mtcnn import MTCNN

Using TensorFlow backend.


In [4]:
sorted(glob.glob('../input/deepfake/meta*'))

['../input/deepfake/metadata0.json',
 '../input/deepfake/metadata1.json',
 '../input/deepfake/metadata10.json',
 '../input/deepfake/metadata11.json',
 '../input/deepfake/metadata12.json',
 '../input/deepfake/metadata13.json',
 '../input/deepfake/metadata14.json',
 '../input/deepfake/metadata15.json',
 '../input/deepfake/metadata16.json',
 '../input/deepfake/metadata17.json',
 '../input/deepfake/metadata18.json',
 '../input/deepfake/metadata19.json',
 '../input/deepfake/metadata2.json',
 '../input/deepfake/metadata20.json',
 '../input/deepfake/metadata21.json',
 '../input/deepfake/metadata22.json',
 '../input/deepfake/metadata23.json',
 '../input/deepfake/metadata24.json',
 '../input/deepfake/metadata25.json',
 '../input/deepfake/metadata26.json',
 '../input/deepfake/metadata27.json',
 '../input/deepfake/metadata28.json',
 '../input/deepfake/metadata29.json',
 '../input/deepfake/metadata3.json',
 '../input/deepfake/metadata30.json',
 '../input/deepfake/metadata31.json',
 '../input/deepf

# Load Train Data

In [5]:
df_train0 = pd.read_json('../input/deepfake/metadata0.json')
df_train1 = pd.read_json('../input/deepfake/metadata1.json')
df_train2 = pd.read_json('../input/deepfake/metadata2.json')
df_train3 = pd.read_json('../input/deepfake/metadata3.json')
df_train4 = pd.read_json('../input/deepfake/metadata4.json')
df_train5 = pd.read_json('../input/deepfake/metadata5.json')
df_train6 = pd.read_json('../input/deepfake/metadata6.json')
df_train7 = pd.read_json('../input/deepfake/metadata7.json')
df_train8 = pd.read_json('../input/deepfake/metadata8.json')
df_train9 = pd.read_json('../input/deepfake/metadata9.json')
df_train10 = pd.read_json('../input/deepfake/metadata10.json')
df_train11 = pd.read_json('../input/deepfake/metadata11.json')
df_train12 = pd.read_json('../input/deepfake/metadata12.json')
df_train13 = pd.read_json('../input/deepfake/metadata13.json')
df_train14 = pd.read_json('../input/deepfake/metadata14.json')
df_train15 = pd.read_json('../input/deepfake/metadata15.json')
df_train16 = pd.read_json('../input/deepfake/metadata16.json')
df_train17 = pd.read_json('../input/deepfake/metadata17.json')
df_train18 = pd.read_json('../input/deepfake/metadata18.json')
df_train19 = pd.read_json('../input/deepfake/metadata19.json')
df_train20 = pd.read_json('../input/deepfake/metadata20.json')
df_train21 = pd.read_json('../input/deepfake/metadata21.json')
df_train22 = pd.read_json('../input/deepfake/metadata22.json')
df_train23 = pd.read_json('../input/deepfake/metadata23.json')
df_train24 = pd.read_json('../input/deepfake/metadata24.json')
df_train25 = pd.read_json('../input/deepfake/metadata25.json')
df_train26 = pd.read_json('../input/deepfake/metadata26.json')
df_train27 = pd.read_json('../input/deepfake/metadata27.json')
df_train28 = pd.read_json('../input/deepfake/metadata28.json')
df_train29 = pd.read_json('../input/deepfake/metadata29.json')
df_train30 = pd.read_json('../input/deepfake/metadata30.json')
df_train31 = pd.read_json('../input/deepfake/metadata31.json')
df_train32 = pd.read_json('../input/deepfake/metadata32.json')
df_train33 = pd.read_json('../input/deepfake/metadata33.json')
df_train34 = pd.read_json('../input/deepfake/metadata34.json')
df_train35 = pd.read_json('../input/deepfake/metadata35.json')
df_train36 = pd.read_json('../input/deepfake/metadata36.json')
df_train37 = pd.read_json('../input/deepfake/metadata37.json')
df_train38 = pd.read_json('../input/deepfake/metadata38.json')
df_train39 = pd.read_json('../input/deepfake/metadata39.json')
df_train40 = pd.read_json('../input/deepfake/metadata40.json')
df_train41 = pd.read_json('../input/deepfake/metadata41.json')
df_val1 = pd.read_json('../input/deepfake/metadata42.json')
df_val2 = pd.read_json('../input/deepfake/metadata43.json')
df_val3 = pd.read_json('../input/deepfake/metadata44.json')
LABELS = ['REAL','FAKE']
df_trains = [df_train0 ,df_train1, df_train2, df_train3, df_train4,
             df_train5, df_train6, df_train7, df_train8, df_train9,df_train10,
            df_train11, df_train12, df_train13, df_train14, df_train15,df_train16, 
            df_train17, df_train18, df_train19, df_train20, df_train21, df_train22, 
            df_train23, df_train24, df_train25, df_train26, df_train27, df_train28, 
            df_train29, df_train30, df_train31, df_train32, df_train33, df_train34,
            df_train34, df_train35, df_train36, df_train37, df_train38, df_train39,
            df_train40, df_train41]
df_vals=[df_val1, df_val2, df_val3]
nums = list(range(len(df_trains)+1))
val_nums=[42, 43, 44]

# Get Paths

Explanation:
The basic idea behind this is to get the path and y first, apply underbalancing, then read X. The reason why I did this in such way is to avoid large RAM usage.

In [6]:
def get_path(num,x):
    num=str(num)
    if len(num)==2:
        path='../input/deepfake/DeepFake'+num+'/DeepFake'+num+'/' + x.replace('.mp4', '') + '.jpg'
    else:
        path='../input/deepfake/DeepFake0'+num+'/DeepFake0'+num+'/' + x.replace('.mp4', '') + '.jpg'
    if not os.path.exists(path):
       raise Exception
    return path
paths=[]
y=[]
for df_train,num in tqdm(zip(df_trains,nums),total=len(df_trains)):
    images = list(df_train.columns.values)
    for x in images:
        try:
            paths.append(get_path(num,x))
            y.append(LABELS.index(df_train[x]['label']))
        except Exception as err:
            #print(err)
            pass

val_paths=[]
val_y=[]
for df_val,num in tqdm(zip(df_vals,val_nums),total=len(df_vals)):
    images = list(df_val.columns.values)
    for x in images:
        try:
            val_paths.append(get_path(num,x))
            val_y.append(LABELS.index(df_val[x]['label']))
        except Exception as err:
            #print(err)
            pass

# Apply Underbalancing Techinique

In [7]:
print('There are '+str(y.count(1))+' fake train samples')
print('There are '+str(y.count(0))+' real train samples')
print('There are '+str(val_y.count(1))+' fake val samples')
print('There are '+str(val_y.count(0))+' real val samples')

There are 64773 fake train samples
There are 12130 real train samples
There are 6048 fake val samples
There are 1261 real val samples


The data is not balanced. We are going to use the undersampling technique.

In [8]:
import random
real=[]
fake=[]
for m,n in zip(paths,y):
    if n==0:
        real.append(m)
    else:
        fake.append(m)
fake=random.sample(fake,len(real))
paths,y=[],[]
for x in real:
    paths.append(x)
    y.append(0)
for x in fake:
    paths.append(x)
    y.append(1)

In [9]:
real=[]
fake=[]
for m,n in zip(val_paths,val_y):
    if n==0:
        real.append(m)
    else:
        fake.append(m)
fake=random.sample(fake,len(real))
val_paths,val_y=[],[]
for x in real:
    val_paths.append(x)
    val_y.append(0)
for x in fake:
    val_paths.append(x)
    val_y.append(1)

In [10]:
print('There are '+str(y.count(1))+' fake train samples')
print('There are '+str(y.count(0))+' real train samples')
print('There are '+str(val_y.count(1))+' fake val samples')
print('There are '+str(val_y.count(0))+' real val samples')

There are 12130 fake train samples
There are 12130 real train samples
There are 1261 fake val samples
There are 1261 real val samples


Now, the data is balanced.

# Read Images

In [11]:
def read_img(path):
    return cv2.cvtColor(cv2.imread(path),cv2.COLOR_BGR2RGB)
X=[]
for img in tqdm(paths):
    X.append(read_img(img))
val_X=[]
for img in tqdm(val_paths):
    val_X.append(read_img(img))

In [12]:
import random
def shuffle(X,y):
    new_train=[]
    for m,n in zip(X,y):
        new_train.append([m,n])
    random.shuffle(new_train)
    X,y=[],[]
    for x in new_train:
        X.append(x[0])
        y.append(x[1])
    return X,y

In [13]:
X,y=shuffle(X,y)
val_X,val_y=shuffle(val_X,val_y)

# Define Model

In [14]:
def InceptionLayer(a, b, c, d):
    def func(x):
        x1 = Conv2D(a, (1, 1), padding='same', activation='elu')(x)
        
        x2 = Conv2D(b, (1, 1), padding='same', activation='elu')(x)
        x2 = Conv2D(b, (3, 3), padding='same', activation='elu')(x2)
            
        x3 = Conv2D(c, (1, 1), padding='same', activation='elu')(x)
        x3 = Conv2D(c, (3, 3), dilation_rate = 2, strides = 1, padding='same', activation='elu')(x3)
        
        x4 = Conv2D(d, (1, 1), padding='same', activation='elu')(x)
        x4 = Conv2D(d, (3, 3), dilation_rate = 3, strides = 1, padding='same', activation='elu')(x4)
        y = Concatenate(axis = -1)([x1, x2, x3, x4])
            
        return y
    return func
    
def define_model(shape=(256,256,3)):
    x = Input(shape = shape)
    
    x1 = InceptionLayer(1, 4, 4, 2)(x)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
    
    x2 = InceptionLayer(2, 4, 4, 2)(x1)
    x2 = BatchNormalization()(x2)        
    x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)        
        
    x3 = Conv2D(16, (5, 5), padding='same', activation = 'elu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
    x4 = Conv2D(16, (5, 5), padding='same', activation = 'elu')(x3)
    x4 = BatchNormalization()(x4)
    if shape==(256,256,3):
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
    else:
        x4 = MaxPooling2D(pool_size=(2, 2), padding='same')(x4)
    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y = LeakyReLU(alpha=0.1)(y)
    y = Dropout(0.5)(y)
    y = Dense(1, activation = 'sigmoid')(y)
    model=Model(inputs = x, outputs = y)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer=Adam(lr=1e-4))
    #model.summary()
    return model
df_model=define_model()
df_model.load_weights('../input/meso-pretrain/MesoInception_DF')
f2f_model=define_model()
f2f_model.load_weights('../input/meso-pretrain/MesoInception_F2F')

This model is the almost same as MesoNet.

# Train Model

In [15]:
from keras.callbacks import LearningRateScheduler
lrs=[1e-3,5e-4,1e-4]
def schedule(epoch):
    return lrs[epoch]

In [16]:
# import keras.backend as K
import gc
kfolds=5
df_models=[]
f2f_models=[]
losses=[]
i=0
while len(df_models)<kfolds:
    model=define_model((150,150,3))
    if i==0:
        model.summary()
    #model.load_weights('../input/meso-pretrain/MesoInception_DF')
    for new_layer, layer in zip(model.layers[1:-8], df_model.layers[1:-8]):
        new_layer.set_weights(layer.get_weights())
    model.fit([X],[y],epochs=3,callbacks=[LearningRateScheduler(schedule)])
    pred=model.predict([val_X])
    l=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    loss=l(val_y,pred)
    losses.append(loss)
    print('fold '+str(i)+' model loss: '+str(loss))
    if loss<0.68:
        df_models.append(model)
    else:
        print('loss too bad, retrain!')
    K.clear_session()
    del model
    gc.collect()
    i+=1
i=0
while len(f2f_models)<kfolds:
    model=define_model((150,150,3))
    #model.load_weights('../input/meso-pretrain/MesoInception_DF')
    for new_layer, layer in zip(model.layers[1:-8], f2f_model.layers[1:-8]):
        new_layer.set_weights(layer.get_weights())
    model.fit([X],[y],epochs=3,callbacks=[LearningRateScheduler(schedule)])
    pred=model.predict([val_X])
    l=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    loss = loss(val_y, pred)
    losses.append(loss)
    print('fold '+str(i)+' model loss: '+str(loss))
    if loss<0.66:
        f2f_models.append(model)
    else:
        print('loss too bad, retrain!')
    K.clear_session()
    del model
    gc.collect()
    i+=1

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 150, 150, 4)  16          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 150, 150, 4)  16          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 150, 150, 2)  8           input_3[0][0]                    
____________________________________________________________________________________________

ValueError: logits and labels must have the same shape ((2522, 1) vs (2522,))

Explanation:
Because of the smaller input size, this code:
```
    for new_layer, layer in zip(model.layers[1:-8], f2f_model.layers[1:-8]):
        new_layer.set_weights(layer.get_weights())
```
fetches only the conv layers weight and apply it onto our model.

In [17]:
models=f2f_models+df_models

In [18]:
def prediction_pipline(X,two_times=False):
    preds=[]
    for model in tqdm(models):
        pred=model.predict([X])
        preds.append(pred)
    preds=sum(preds)/len(preds)
    if two_times:
        return larger_range(preds,2)
    else:
        return preds
def larger_range(model_pred,time):
    return (((model_pred-0.5)*time)+0.5)

In [19]:
best_model_pred=models[losses.index(min(losses))].predict([val_X])

ValueError: min() arg is an empty sequence

In [20]:
model_pred=prediction_pipline(val_X)

ZeroDivisionError: division by zero

# Validation

Some baselines:

In [21]:
random_pred=np.random.random(len(val_X))
allone_pred=np.array([1 for _ in range(len(val_X))])
allzero_pred=np.array([0 for _ in range(len(val_X))])
allpoint5_pred=np.array([0.5 for _ in range(len(val_X))])


Model loss

In [22]:
l=tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [23]:

if l(val_y,model_pred.clip(0.35,0.65))<l(val_y,larger_range(model_pred,2).clip(0.35,0.65)):
    two_times=False
    print('simple averaging is better')
else:
    two_times=True
    print('two times larger range is better')
two_times=False #This is not a bug. I did this intentionally because the model can't get most of the private validation set right(based on LB)

NameError: name 'model_pred' is not defined

According to validation loss, averaging is better than best single model.

Take a look at predictions

In [24]:
import scipy
print(model_pred.clip(0.35,0.65).mean())
print(scipy.stats.median_absolute_deviation(model_pred.clip(0.35,0.65))[0])

NameError: name 'model_pred' is not defined

In [25]:
def check_answers(pred,real,num):
    for i,(x,y) in enumerate(zip(pred,real)):
        correct_incorrect='correct ✅ ' if round(float(x),0)==round(float(y),0) else 'incorrect❌'
        print(correct_incorrect+' prediction: '+str(x[0])+', answer: '+str(y))
        if i>num:
            return
def correct_precentile(pred,real):
    correct=0
    incorrect=0
    for x,y in zip(pred,real):
        if round(float(x),0)==round(float(y),0):
            correct+=1
        else:
            incorrect+=1
    print('number correct: '+str(correct)+', number incorrect: '+str(incorrect))
    print(str(round(correct/len(real)*100,1))+'% correct'+', '+str(round(incorrect/len(real)*100,1))+'% incorrect')
check_answers(model_pred,val_y,15)
correct_precentile(model_pred,val_y)

NameError: name 'model_pred' is not defined

In [26]:
del X,y,val_X,val_y

# Make submission

In [27]:
MAX_SKIP=10
NUM_FRAME=150
test_dir = '/kaggle/input/deepfake-detection-challenge/test_videos/'
filenames = os.listdir(test_dir)
prediction_filenames = filenames
test_video_files = [test_dir + x for x in filenames]
detector = MTCNN()
def detect_face(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    final = []
    detected_faces_raw = detector.detect_faces(img)
    if detected_faces_raw==[]:
        #print('no faces found')
        return []
    confidences=[]
    for n in detected_faces_raw:
        x,y,w,h=n['box']
        final.append([x,y,w,h])
        confidences.append(n['confidence'])
    if max(confidences)<0.9:
        return []
    max_conf_coord=final[confidences.index(max(confidences))]
    #return final
    return max_conf_coord
def crop(img,x,y,w,h):
    x-=40
    y-=40
    w+=80
    h+=80
    if x<0:
        x=0
    if y<=0:
        y=0
    return cv2.cvtColor(cv2.resize(img[y:y+h,x:x+w],(150,150)),cv2.COLOR_BGR2RGB)
def detect_video(video):
    v_cap = cv2.VideoCapture(video)
    v_cap.set(1, NUM_FRAME)
    success, vframe = v_cap.read()
    vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
    bounding_box=detect_face(vframe)
    if bounding_box==[]:
        count=0
        current=NUM_FRAME
        while bounding_box==[] and count<MAX_SKIP:
            current+=1
            v_cap.set(1,current)
            success, vframe = v_cap.read()
            vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
            bounding_box=detect_face(vframe)
            count+=1
        if bounding_box==[]:
            print('no faces found')
            prediction_filenames.remove(video.replace('/kaggle/input/deepfake-detection-challenge/test_videos/',''))
            return None
    x,y,w,h=bounding_box
    v_cap.release()
    return crop(vframe,x,y,w,h)
test_X = []
for video in tqdm(test_video_files):
    x=detect_video(video)
    if x is None:
        continue
    test_X.append(x)

no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found
no faces found



In [28]:
df_test=pd.read_csv('/kaggle/input/deepfake-detection-challenge/sample_submission.csv')
df_test['label']=0.5
preds=prediction_pipline(test_X,two_times=two_times).clip(0.35,0.65)
for pred,name in zip(preds,prediction_filenames):
    name=name.replace('/kaggle/input/deepfake-detection-challenge/test_videos/','')
    df_test.iloc[list(df_test['filename']).index(name),1]=pred

NameError: name 'two_times' is not defined

In [29]:
print(preds.clip(0.35,0.65).mean())
print(scipy.stats.median_absolute_deviation(preds.clip(0.35,0.65))[0])
print(preds[:10])

NameError: name 'preds' is not defined

In [30]:
df_test.head()

,filename,label
0,aassnaulhq.mp4,0.5
1,aayfryxljh.mp4,0.5
2,acazlolrpz.mp4,0.5
3,adohdulfwb.mp4,0.5
4,ahjnxtiamx.mp4,0.5


In [31]:
df_test.to_csv('submission.csv',index=False)

# Further Work
1. Do some hyperparamater tuning
2. Train on the whole video(and maybe also sound)
3. Try LSTM-CNN